In [ ]:
!pip -q install -U transformers datasets accelerate scikit-learn pandas

import numpy as np, pandas as pd, torch, os
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, set_seed
from sklearn.metrics import accuracy_score, f1_score

# ====== PATHS (change if needed) ======
TRAIN_CSV = "/content/train_sent_emo.csv"
VAL_CSV   = "/content/dev_sent_emo.csv"
TEST_CSV  = "/content/test_sent_emo.csv"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 144.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 142.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 50.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


In [ ]:
# =======================
# MELD (Ekman-7) BERT fine-tune (5 seeds)
# - best checkpoint selected by weighted_f1 on VAL
# - evaluate on TEST
# =======================


# ====== CONFIG ======
MODEL_BASE = "distilbert-base-cased"
TEXT_COL = "Utterance"
LABEL_COL = "Emotion"

LABELS = ["neutral", "joy", "sadness", "anger", "surprise", "fear","disgust"]
label2id = {l:i for i,l in enumerate(LABELS)}
id2label = {i:l for l,i in label2id.items()}

SEEDS = [42,43,44,45,46]
LR = 2e-5
EPOCHS = 4
BATCH_TRAIN = 16
BATCH_EVAL  = 32
MAX_LEN = 256
OUT_ROOT = "bert_large_meld"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tok = AutoTokenizer.from_pretrained(MODEL_BASE, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [ ]:
# ====== DATA ======
def load_df(path):
    df = pd.read_csv(path).dropna(subset=[TEXT_COL, LABEL_COL]).copy()
    df[TEXT_COL]  = df[TEXT_COL].astype(str)
    df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip().str.lower()
    df = df[df[LABEL_COL].isin(LABELS)].copy()
    return df

def to_ds(df):
    ds = Dataset.from_pandas(df[[TEXT_COL, LABEL_COL]], preserve_index=False)
    def enc(batch):
        out = tok(batch[TEXT_COL], truncation=True, padding=False, max_length=MAX_LEN)
        out["labels"] = [label2id[x] for x in batch[LABEL_COL]]
        return out
    return ds.map(enc, batched=True, remove_columns=[TEXT_COL, LABEL_COL])

train_ds = to_ds(load_df(TRAIN_CSV))
val_ds   = to_ds(load_df(VAL_CSV))
test_ds  = to_ds(load_df(TEST_CSV))

print("Counts:", len(train_ds), len(val_ds), len(test_ds))

# ====== METRICS ======
def compute_metrics(eval_pred):
    logits, y_true = eval_pred
    y_pred = np.argmax(logits, axis=1)
    return {
        "acc": accuracy_score(y_true, y_pred),
        "weighted_f1": f1_score(y_true, y_pred, average="weighted"),
        "macro_f1": f1_score(y_true, y_pred, average="macro"),
    }

Map:   0%|          | 0/9989 [00:00<?, ? examples/s]

Map:   0%|          | 0/1109 [00:00<?, ? examples/s]

Map:   0%|          | 0/2610 [00:00<?, ? examples/s]

Counts: 9989 1109 2610


In [ ]:
rows = []
best_ckpts = {}  # seed -> checkpoint path

for seed in SEEDS:
    print("\n" + "="*20, "SEED", seed, "="*20)
    set_seed(seed)

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_BASE,
        num_labels=len(LABELS),
        label2id=label2id,
        id2label=id2label
    ).to(DEVICE)

    args = TrainingArguments(
        output_dir=f"{OUT_ROOT}_seed{seed}",
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="weighted_f1",
        greater_is_better=True,

        learning_rate=LR,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_TRAIN,
        per_device_eval_batch_size=BATCH_EVAL,
        weight_decay=0.01,
        warmup_ratio=0.06,

        fp16=torch.cuda.is_available(),
        report_to="none",
        seed=seed,
        logging_steps=50,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tok,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    best_ckpts[seed] = trainer.state.best_model_checkpoint
    res = trainer.evaluate(test_ds)

    rows.append({
        "seed": seed,
        "test_acc": float(res["eval_acc"]),
        "test_weighted_f1": float(res["eval_weighted_f1"]),
        "test_macro_f1": float(res["eval_macro_f1"]),
        "best_ckpt": best_ckpts[seed],
    })

df = pd.DataFrame(rows)
print("\nPer-seed results:")
display(df)

print("\nMEAN:")
display(df.drop(columns=["seed","best_ckpt"]).mean().to_frame("mean"))

print("\nSTD:")
display(df.drop(columns=["seed","best_ckpt"]).std().to_frame("std"))



==================== SEED 42 ====================


model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1128718231.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Acc,Weighted F1,Macro F1
1,1.072500,1.207034,0.589720,0.550291,0.355298
2,1.072300,1.177748,0.594229,0.565259,0.374501
3,0.905800,1.218614,0.596934,0.563401,0.379243
4,0.712100,1.251536,0.593327,0.568707,0.395375



==================== SEED 43 ====================


...

,seed,test_acc,test_weighted_f1,test_macro_f1,best_ckpt
0,42,0.632950,0.613811,0.402753,bert_large_meld_seed42/checkpoint-2500
1,43,0.637165,0.612667,0.393243,bert_large_meld_seed43/checkpoint-1875
2,44,0.634866,0.609661,0.394790,bert_large_meld_seed44/checkpoint-1875
3,45,0.640996,0.614641,0.395878,bert_large_meld_seed45/checkpoint-1875
4,46,0.630651,0.611136,0.393392,bert_large_meld_seed46/checkpoint-2500



MEAN:


,mean
test_acc,0.635326
test_weighted_f1,0.612383
test_macro_f1,0.396011



STD:


,std
test_acc,0.003976
test_weighted_f1,0.002011
test_macro_f1,0.003921


In [ ]:
import os, shutil

for seed, ckpt_path in best_ckpts.items():
    best_dir = f"{OUT_ROOT}_seed{seed}_BEST"
    if os.path.exists(best_dir):
        shutil.rmtree(best_dir)
    shutil.copytree(ckpt_path, best_dir)
    tok.save_pretrained(best_dir)
    print(f"Saved BEST for seed {seed}: {best_dir}")


NameError: name 'best_ckpts' is not defined